# Entity Matching

The process of finding matching records in tables is called entity matching. It can be used for deduplication of tables or to link tables to merge the information that they contain. The question that we want to answer in this notebook is whether machine learning has something to add to traditional programming methods for entity matching and whether fancy deep learning approaches really outperform simpler machine learning models that have already been around for decades.

To investigate this, we make use of two existing entity matching packages: the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) (PRLT), which offers support for traditional programimng techniques and basic machine learning models and [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/), a package that adopts deep learning techniques. We test both packages on  scientific bibliographic data from [ACM](https://www.acm.org/) and [DBLP](https://dblp.org/), which we downloaded from the [Database Group Leipzig ](https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution). This is an independent dataset that was not used to advertise either of the two methods.

## Package Imports

We now import the packages that we are going to use, together with our self-created module em_helper that contains some utitility functions to generate and display classification performance metrics.

In [1]:
try:
   import recordlinkage
except:
    !pip install -q recordlinkage
import pandas as pd
import numpy as np
import os
try:
  import deepmatcher as dm
except:
    !pip install -q deepmatcher

## Setting random seeds
We would like to keep our experiment as reproducible a possible so we start by fixing the random seed.

In [2]:
np.random.seed(0)

## Mounting Google Drive
To get access to data, we will mount our Google Drive (comment this out when you are working on a local machine).

In [3]:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


# Setting root directory
We now set the root directory. Note that this differs when you are running from Google Colab compared to running locally. Comment out to non-appliying option.

In [4]:
# root = '.'
root = root = '/drive/My Drive/EntityMatching/'
import sys 
sys.path.append(root)

## Data Loading
Having all our dependencies in place, we now load and display the two tables that contain the records that we want to match.

In [6]:
data_left = os.path.join(root, 'Data/DBLP2.csv')
data_right = os.path.join(root, 'Data/ACM.csv')
left_df = pd.read_csv(data_left, encoding='cp1252')
right_df = pd.read_csv(data_right, encoding='cp1252')
left_df.set_index('id',inplace=True)
right_df.set_index('id',inplace=True)
display(left_df)
display(right_df)

,title,authors,venue,year
id,,,,
journals/sigmod/Mackay99,Semantic Integration of Environmental Models f...,D. Scott Mackay,SIGMOD Record,1999
conf/vldb/PoosalaI96,Estimation of Query-Result Distribution and it...,"Viswanath Poosala, Yannis E. Ioannidis",VLDB,1996
conf/vldb/PalpanasSCP02,Incremental Maintenance for Non-Distributive A...,"Themistoklis Palpanas, Richard Sidle, Hamid Pi...",VLDB,2002
conf/vldb/GardarinGT96,Cost-based Selection of Path Expression Proces...,"Zhao-Hui Tang, Georges Gardarin, Jean-Robert G...",VLDB,1996
conf/vldb/HoelS95,Benchmarking Spatial Join Operations with Spat...,"Erik G. Hoel, Hanan Samet",VLDB,1995
...,...,...,...,...
journals/tods/KarpSP03,A simple algorithm for finding frequent elemen...,"Scott Shenker, Christos H. Papadimitriou, Rich...",ACM Trans. Database Syst.,2003
conf/vldb/LimWV03,SASH: A Self-Adaptive Histogram Set for Dynami...,"Lipyeow Lim, Min Wang, Jeffrey Scott Vitter",VLDB,2003
journals/tods/ChakrabartiKMP02,Locally adaptive dimensionality reduction for ...,"Kaushik Chakrabarti, Eamonn J. Keogh, Michael ...",ACM Trans. Database Syst.,2002


,title,authors,venue,year
id,,,,
304586,The WASA2 object-oriented workflow management ...,"Gottfried Vossen, Mathias Weske",International Conference on Management of Data,1999
304587,A user-centered interface for querying distrib...,"Isabel F. Cruz, Kimberly M. James",International Conference on Management of Data,1999
304589,"World Wide Database-integrating the Web, CORBA...","Athman Bouguettaya, Boualem Benatallah, Lily H...",International Conference on Management of Data,1999
304590,XML-based information mediation with MIX,"Chaitan Baru, Amarnath Gupta, Bertram Lud&#228...",International Conference on Management of Data,1999
304582,The CCUBE constraint object-oriented database ...,"Alexander Brodsky, Victor E. Segal, Jia Chen, ...",International Conference on Management of Data,1999
...,...,...,...,...
672977,Dual-Buffering Strategies in Object Bases,"Alfons Kemper, Donald Kossmann",Very Large Data Bases,1994
950482,Guest editorial,"Philip A. Bernstein, Yannis Ioannidis, Raghu R...",The VLDB Journal &mdash; The International Jou...,2003
672980,GraphDB: Modeling and Querying Graphs in Datab...,Ralf Hartmut G&#252;ting,Very Large Data Bases,1994


# Defining Candidate Links
To translate the matching problem into a standard classification problem, we first form all possible combinations of rows in the DBLP table and rows in the ACM table. In other words, we form the Cartesian product of the indices of these tables. The DBLP table has 2616 rows and the the ACM table has 2294 rows, so this results in a collection of 2616 x 2294 = 6001104 combinations. 

The size of this set of possible links now places us for a dilemma:
1. We would like to use as much of the data as possible to reach a sound conclusion.
2. DeepMatcher is not that fast and will take ages to process such a  large dataset.

To solve this dilemma, we apply a technique that is commonly used in entity matching: _blocking_. By using a simple heuristic, we filter out row combinations, of which we can be (almost) certain that they do not match. By blocking these row combinations, we end up with a considerably smaller set of candidate links that is much easier to manage. 

To perform blocking, we make use of the string comparison capabilities of the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/). We block all combinations with a [Lehvenstein-distance](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance)-based similarity measure that is smaller than 0.3, where 0 indicates a complete mismatch and 1 indicates identical strings. By doing this, we reduce the number of candidate links to 162894, which amounts to a data reduction of about 97%.

In [7]:
# Full link
indexer = recordlinkage.Index()
indexer.full()
links = indexer.index(left_df, right_df)

# Comparison step for blocking
compare_cl = recordlinkage.Compare()
compare_cl.string('title', 'title', threshold=0.3, label='title')
features = compare_cl.compute(links, left_df, right_df)

# Blocking
candidate_links = links[features.sum(axis=1) > 0]

# Show data reduction
display("Number of candidate links in Cartesian product: {}".format(len(links)))
display("Number of candidate links after blocking: {}".format(len(candidate_links)))

'Number of candidate links in Cartesian product: 6001104'

'Number of candidate links after blocking: 162894'

## Inspection of blocking effects
Since blocking might have the unwanted effect of throwing away a considerable number of true mtaches, we briefly verify whether this is the case. We load the data set containing the true links and see how much of these links are still present in the blocked data set. It turns out that only 8 of the 2224 true matches are discarded by the blocking procesdure, which amounts to 0.36%. This seems to be a defendable sacrifice to make for the 97% data reduction that is achieved by blocking.

In [8]:
data_match = os.path.join(root, 'Data/DBLP-ACM_perfectMapping.csv')
matches = pd.read_csv(data_match, encoding='cp1252')
matches_tuples = list(matches.itertuples(index=False, name=None)) 
candidate_links_tuples = candidate_links.tolist()
intersection = set(candidate_links_tuples).intersection(matches_tuples)
display("Number of true matches in full set: {}".format(len(intersection)))
display("Number of true matches in blocked set: {}".format(len(matches_tuples)))

'Number of true matches in full set: 2216'

'Number of true matches in blocked set: 2224'

## Data preparation for DeepMatcher
To use the DeepMatcher classifier, we need to transform our data into something that [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) can understand. DeepMatcher takes in a table with record pairs of the two tables, which need to be matched. The records of the first table  are marked by adding the prefix "left_" to their feature names, and the records of the second table are marked by adding the prefix "right_". The target variable that indicates whether a pair forms a match is called "label" and takes on the values 0 (no match) or 1 (match). To prevent unwanted correlations between the subsequent rows, we additionally shuffle the table.

In [9]:
left_df_renamed =left_df.reset_index().add_prefix('left_')
left_df_renamed['join'] = 1
right_df_renamed =right_df.reset_index().add_prefix('right_')
right_df_renamed['join'] = 1
final_df_full = pd.merge(left_df_renamed, right_df_renamed, on="join")
final_df_full.insert(0,'label',0)
final_df_full['combined_index'] = list(zip(final_df_full.left_id, final_df_full.right_id))
final_df_full.loc[final_df_full['combined_index'].isin(matches_tuples),'label'] = 1
final_df_blocked = final_df_full.loc[final_df_full['combined_index'].isin(candidate_links)]
final_df = final_df_blocked.drop(columns=['combined_index','left_id','right_id','join'])
final_df.index.name = 'id'
final_df = final_df.sample(frac=1)
display(final_df)

,label,left_title,left_authors,left_venue,left_year,right_title,right_authors,right_venue,right_year
id,,,,,,,,,
5962337,0,Indexing Images in Oracle8i,"Melliyal Annamalai, Samuel DeFazio, Rajiv Chopra",SIGMOD Conference,2000,Standards in practice,"Andrew Eisenberg, Jim Melton",ACM SIGMOD Record,1998
3063086,0,Illuminating the Dark Side of Web Services,Michael L. Brodie,VLDB,2003,Information warfare and security,H. V. Jagadish,ACM SIGMOD Record,2001
2193273,0,Locating and accessing data repositories with ...,"George A. Mihaila, Louiqa Raschid, Anthony Tom...",VLDB J.,2002,Logical logging to extend recovery to new domains,"David Lomet, Mark Tuttle",International Conference on Management of Data,1999
5707520,1,Storing Semistructured Data with STORED,"Alin Deutsch, Mary F. Fernandez, Dan Suciu",SIGMOD Conference,1999,Storing semistructured data with STORED,"Alin Deutsch, Mary Fernandez, Dan Suciu",International Conference on Management of Data,1999
4100816,0,From Structured Documents to Novel Query Facil...,"Sophie Cluet, Michel Scholl, Serge Abiteboul, ...",SIGMOD Conference,1994,The Fittest Survives: An Adaptive Approach to ...,"Hongjun Lu, Kian-Lee Tan, Son Dao",Very Large Data Bases,1995
...,...,...,...,...,...,...,...,...,...
3597271,0,Parallel Evaluation of Multi-Join Queries,"Jan Flokstra, Annita N. Wilschut, Peter M. G. ...",SIGMOD Conference,1995,Orthogonal optimization of subqueries and aggr...,"C&#233;sar Galindo-Legaria, Milind Joshi",International Conference on Management of Data,2001
3545376,0,Dynamic Information Visualization,Yannis E. Ioannidis,SIGMOD Record,1996,MineSet(tm): A System for High-End Data Mining...,NaN,Very Large Data Bases,1996
5587905,0,A Language Based Multidatabase System,"Konrad Schwarz, eva Kühn, Thomas Tschernko",SIGMOD Conference,1994,Transparent mid-tier database caching in SQL s...,"Per-&#197;ke Larson, Jonathan Goldstein, Jingr...",International Conference on Management of Data,2003


## Creating Sets for Training, Validation and Testing
Now that our dataset is the right format, we create a train, validation and test set by applying a 60:20:20 split. The resulting datasets are saved to csv-files inside the Data directory.

In [10]:
train_split,validate_split,test_split = np.split(final_df, [int(.6 * len(final_df)), int(.8 * len(final_df))])
train_file = os.path.join(root,'Data/train_block.csv')
validate_file = os.path.join(root,'Data/validate_block.csv')
test_file = os.path.join(root,'Data/test_block.csv')
train_split.to_csv(train_file)
validate_split.to_csv(validate_file)
test_split.to_csv(test_file)

## Deepmatcher Data Preprocessing and Feature Engineering
We now let [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) apply its standard data preprecessing and feature engineering procedure. Due to the time limits of the Project Challenge, we have not delved deeper into this, but rather used it as a black box. To get a rough idea of what [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) does behind the scenes, we display the still human-interpretable raw data table, which holds the result of text normalization.

In [11]:
train, validate, test = dm.data.process(
    path='.',
    left_prefix='left_',
    right_prefix='right_',
    label_attr='label',
    id_attr='id',
    cache=None,
    train=train_file,
    validation=validate_file,
    test=test_file)
train_table = train.get_raw_table()
display(train_table)


Reading and processing data from "/drive/My Drive/EntityMatching/Data/train_block.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "/drive/My Drive/EntityMatching/Data/validate_block.csv"
0% [##############################] 100% | ETA: 00:00:00
Reading and processing data from "/drive/My Drive/EntityMatching/Data/test_block.csv"
0% [##############################] 100% | ETA: 00:00:00
Building vocabulary
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07

Computing principal components
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:01:16


,id,label,left_title,left_authors,left_venue,left_year,right_title,right_authors,right_venue,right_year
0,5962337,0,indexing images in oracle8i,"melliyal annamalai , samuel defazio , rajiv ch...",sigmod conference,2000,standards in practice,"andrew eisenberg , jim melton",acm sigmod record,1998
1,3063086,0,illuminating the dark side of web services,michael l. brodie,vldb,2003,information warfare and security,h. v. jagadish,acm sigmod record,2001
2,2193273,0,locating and accessing data repositories with ...,"george a. mihaila , louiqa raschid , anthony t...",vldb j .,2002,logical logging to extend recovery to new domains,"david lomet , mark tuttle",international conference on management of data,1999
3,5707520,1,storing semistructured data with stored,"alin deutsch , mary f. fernandez , dan suciu",sigmod conference,1999,storing semistructured data with stored,"alin deutsch , mary fernandez , dan suciu",international conference on management of data,1999
4,4100816,0,from structured documents to novel query facil...,"sophie cluet , michel scholl , serge abiteboul...",sigmod conference,1994,the fittest survives : an adaptive approach to...,"hongjun lu , kian-lee tan , son dao",very large data bases,1995
...,...,...,...,...,...,...,...,...,...,...
97731,3346855,0,highly concurrent cache consistency for indice...,"michael j. carey , markos zaharioudakis",sigmod conference,1997,new concurrency control algorithms for accessi...,"v. w. setzer , andrea zisman",very large data bases,1994
97732,5201113,0,indexing large metric spaces for similarity se...,"z. meral özsoyoglu , tolga bozkaya",acm trans . database syst .,1999,estimating page fetches for index scans with f...,"arun swami , k. bernhard schiefer",the vldb journal & mdash ; the international j...,1995
97733,3473534,0,the sr-tree : an index structure for high-dime...,"norio katayama , shin'ichi satoh",sigmod conference,1997,the onion technique : indexing for linear opti...,"yuan-chi chang , lawrence bergman , vittorio c...",international conference on management of data,2000
97734,3464565,0,algorithms for mining association rules for bi...,"kunikazu yoda , hirofumi matsuzawa , takeshi t...",vldb,1998,data model for extensible support of explicit ...,"joan peckham , bonnie mackellar , michael doherty",the vldb journal & mdash ; the international j...,1995


## DeepMatcher Model Training
We are now ready to train our DeepMatcher model. We take over the values of he DeepMatcher ["Getting started"-notebook](https://github.com/anhaidgroup/deepmatcher/blob/master/examples/getting_started.ipynb), but make a couple of small adjustments. Knowing that an epoch will take about half an hour on Google Colab and 2 hours on our local machine, we limit the number of epochs to 8 instead of 10, and since we have not seen any substantial benefit of weighting in smaller experiments, we practically disable it by setting pos_neg_ratio = 1.

The training function prints the performance metric F1-measure, precision and recall for both the training and the validation set at each epoch and uses the F1-measure on the validation set to select the best model. The F1-measure is more suitable than accuracy in thie case, because even in the blocked dataset, the fraction of true matches is very small (about 1.4% ).

In [12]:
model_file = os.path.join(root,'Results/model_dblp_acm_block.pth')
model = dm.MatchingModel(attr_summarizer='hybrid')
f1_dm_val = model.run_train(
    train,
    validate,
    epochs=8,
    batch_size=16,
    best_save_path= model_file,
    pos_neg_ratio=1)

* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2352: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:23:04


Finished Epoch 1 || Run Time: 1140.1 | Load Time:  245.1 || F1:  92.86 | Prec:  95.68 | Rec:  90.20 || Ex/s:  70.56

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:04:11


Finished Epoch 1 || Run Time:  171.0 | Load Time:   80.6 || F1:  97.04 | Prec:  97.52 | Rec:  96.56 || Ex/s: 129.51

* Best F1: tensor(97.0370, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

Finished Epoch 7 || Run Time:  174.0 | Load Time:   80.4 || F1:  98.39 | Prec:  99.50 | Rec:  97.30 || Ex/s: 128.06

* Best F1: tensor(98.3851, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 8


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:22:53


Finished Epoch 8 || Run Time: 1135.0 | Load Time:  239.6 || F1:  99.59 | Prec:  99.62 | Rec:  99.55 || Ex/s:  71.10

===>  EVAL Epoch 8


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:04:08


Finished Epoch 8 || Run Time:  169.6 | Load Time:   79.0 || F1:  98.00 | Prec:  99.49 | Rec:  96.56 || Ex/s: 131.08

---------------------

Loading best model...
Training done.


## DeepMatcher Testing
We are now curious to see what the selected model does on the test set. We see that compared to the training and validation results, there is no serious drop in the F1-value, so we are reasonably confident that our model is not overfitting.

In [13]:
f1_dm_test = model.run_eval(test)

===>  EVAL Epoch 7


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:54


Finished Epoch 7 || Run Time:   89.5 | Load Time:   85.7 || F1:  99.07 | Prec:  99.17 | Rec:  98.96 || Ex/s: 186.03



##  PRLT Feature Engineering
We are now ready to switch to the the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/), which uses a simpler approach to feature engineering. Basically, we fill a vector with zeros of ones that indicate whether a record pair matches with respect to a certain column. So for each of the columns in the left/right tables, we define how similar they need to be to be matching. All the values of the per-column-matches are then concatenated to a global feature vector of zeros and ones, where  an all zero vector means that the pair matches in none of the columns, an an all-one-vector indicates a match with respect to every column. This vector of zeros and ones is then fed to the various classifier models.

To define when we call two colum values matching, we let ourselves be inspired by the [example](https://recordlinkage.readthedocs.io/en/latest/notebooks/link_two_dataframes.html) that the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) provides in its introductory text. Here most of the column matches are defined in terms of a similarity that is based on the [Jaro Winkler distance](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance). The only small adaptation that we make is in the choice of the similarity threshold value for "venue", because a short inspection of the data indicates that matching records tend to agree less with respect to "venue" than they do with respect to "title" or 'author".

In [14]:
# Column-based matching criteria that define the feature vector
compare_cl = recordlinkage.Compare()
compare_cl.string('title', 'title', method='jarowinkler', threshold=0.85,label = 'title')
compare_cl.string('authors', 'authors', method='jarowinkler', threshold=0.85,label = 'authors')
compare_cl.exact('year', 'year', label='year')
compare_cl.string('venue', 'venue', method='jarowinkler', threshold=0.65,label = 'authors')

<Compare>

## PRLT Data Loading and Feature Computation
We now load our previously saved train, test andvalidation sets from file. Ironically, to feed the data to the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) methods, we need to split up the joined table into two separate tables and provide a list of links indicating which record combinations need to be considered in the computation of the feature vectors.

In [17]:
# Load data
true_links = {}
features = {}
validation_dict = {}
datasets= {'train':train_file, 
'test':test_file,
'validation':validate_file}

for key in datasets:
    df = pd.read_csv(datasets[key])
    nof_cols = int((df.shape[1] - 2)/2)
    dfA = df.iloc[:,2:nof_cols + 2] # left table
    dfB = df.iloc[:,nof_cols + 2:df.shape[1]] # right table
    # Delete 'left_' and 'right_' prefixes from column names
    dfA.rename(columns={c:c[5:] for c in dfA.columns },inplace=True) 
    dfB.rename(columns={c:c[6:] for c in dfB.columns },inplace=True)

    tuples = [(i,i) for i in range(len(df)) if df.iloc[i]['label'] == 1]
    true_links[key] = pd.MultiIndex.from_tuples(tuples)

    tuples_full = [(i,i) for i in range(len(df))]
    candidate_links = pd.MultiIndex.from_tuples(tuples_full)
    # Final features (used in all methods)
    features[key] = compare_cl.compute(candidate_links, dfA, dfB)

## PRLT Helper functions
We now define some helper functions used for the computation and printing of the permonce metrics of the PRLT results.

In [28]:
# Some helper functions
def print_results(name, table):
    print("".join(['*' for x in range(len(name) + 1)]))
    print('{}'.format(name))
    print("".join(['*' for x in range(len(name) + 1)]))
    print("Confusion matrix:")
    print(table['confusion_matrix'])
    print("Accuracy: {}".format(table['accuracy']))
    print("Recall: {}".format(table['recall']))
    print("F-score: {}".format(table['f-score']))
    print('\n')
    
def performance_metrics(true_links, result, set_size):
    validation = {}
    validation['confusion_matrix'] = recordlinkage.confusion_matrix(true_links, 
    result, set_size)
    validation['accuracy'] = recordlinkage.accuracy(true_links, result, 
    len(features['validation']))
    validation['precision'] = recordlinkage.precision(true_links, result)
    validation['recall'] = recordlinkage.recall(true_links, result)
    validation['f-score'] = recordlinkage.fscore(true_links, result)
    return validation

## PRLT Model Training
We are now ready to train the various models included in the [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/). There are three super vised methods (Logistic regression, Naive Bayes and Support vector machines) and two unsupervised methods (K-Means and [ECM](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.18.3828&rep=rep1&type=pdf)). We also add a simple heuristic that we label as "Hand-tuned". This basically looks whether records match on the majority of columns (more ones than zeroes in the feature vector) and classifies a pair as a match if this is the case. We assume that this heuristic as a proxy for traditional programming methods as implemented in tools like Power BI, since these are also based on string edit distances. 

The results of the heuristic and the various models on the validation set is displayed below. As we can see, all the supervised  trained models clearly outperform the simple heuristic, whereas the best [Python Record Linkage Toolkit](https://recordlinkage.readthedocs.io/en/latest/) model only slightly underperforms the much more complicated and time-consuming [DeepMatcher](https://anhaidgroup.github.io/deepmatcher/html/) model. Interestingly, the unsupervised ECM model also is a significant gain compared to the simple heuristic, indicating that also without labels, a machine learning approach can be beneficial.

In [29]:
classifiers= {
'Hand-tuned':None,
'Logistic regression':recordlinkage.LogisticRegressionClassifier(),
'Naive Bayes': recordlinkage.NaiveBayesClassifier(),
'Support vector machine': recordlinkage.SVMClassifier(),
'K-means': recordlinkage.KMeansClassifier(),
'ECM': recordlinkage.ECMClassifier()}

for key in classifiers:
    validation_dict[key] = {}
    if key == 'Hand-tuned':
        # Immediate prediction
        result = features['validation'][features['validation'].sum(axis=1) > 2].index
    else:
        # Training 
        if key == 'ECM' or key == 'K-means':
            classifiers[key].fit(features['train']) 
        else:
            classifiers[key].fit(features['train'], true_links['train'])
        # Predict the match status for all test record pairs
        result =  classifiers[key].predict(features['validation'])
        
    # Validate
    validation_dict[key] = performance_metrics(true_links['validation'], result, len(features['validation']))
    
    #Print results
    print_results(key, validation_dict[key])


***********
Hand-tuned
***********
Confusion matrix:
[[  244   163]
 [   27 32145]]
Accuracy: 0.9941680223456828
Recall: 0.5995085995085995
F-score: 0.71976401179941


********************
Logistic regression
********************
Confusion matrix:
[[  392    15]
 [   30 32142]]
Accuracy: 0.9986187421345039
Recall: 0.9631449631449631
F-score: 0.9457177322074789


************
Naive Bayes
************
Confusion matrix:
[[  395    12]
 [   31 32141]]
Accuracy: 0.9986801313729703
Recall: 0.9705159705159705
F-score: 0.9483793517406962


***********************
Support vector machine
***********************
Confusion matrix:
[[  392    15]
 [   30 32142]]
Accuracy: 0.9986187421345039
Recall: 0.9631449631449631
F-score: 0.9457177322074789


********
K-means
********
Confusion matrix:
[[  407     0]
 [ 3467 28705]]
Accuracy: 0.8935817551183277
Recall: 1.0
F-score: 0.190142490072413


****
ECM
****
Confusion matrix:
[[  400     7]
 [   85 32087]]
Accuracy: 0.9971760950305412
Recall: 0.982800982

## PRLT Testing
We select the best model based on the F1-measure on the vaildation set and give it the test set to classify. The result is displayed below. Notice that the result from DeepMatcher is only slightly better.

In [32]:
f_scores = {key:validation_dict[key]['f-score'] for key in validation_dict}
best_model = max(f_scores, key = f_scores.get) 
if best_model == 'Hand-tuned':
    result = features['test'][features['test'].sum(axis=1) > 2].index
else:
    result = classifiers[best_model].predict(features['test'])
test_dict =  performance_metrics(true_links['test'], result, len(features['test']))
print_results("Selected model ({}) on test set".format(best_model), test_dict)

*****************************************
Selected model (Naive Bayes) on test set
*****************************************
Confusion matrix:
[[  464    19]
 [   28 32068]]
Accuracy: 0.9985573528960373
Recall: 0.9606625258799172
F-score: 0.9517948717948718




## Result summary
To make a comparison between the models easier, we print a table with all the F1-measures.

In [36]:
model_names = list(classifiers.keys())
data = {'Model': model_names + ['DeepMatcher'],
        'F1 validation': [validation_dict[n]['f-score'] for n in model_names] + [f1_dm_val.item()/100],
       'F1 test':[test_dict['f-score'] if n == best_model else '' for n in model_names] + [f1_dm_test.item()/100]}
df_summary = pd.DataFrame (data,columns = ['Model','F1 validation','F1 test'])
df_summary.set_index('Model', inplace=True)
model_selection = ['DeepMatcher' ,best_model]
df_summary = df_summary.style.apply(lambda x: ['background:lightgreen' if x in model_selection else 'background:white' for x in df_summary.index])
display(df_summary)

,F1 validation,F1 test
Model,,
Hand-tuned,0.719764,
Logistic regression,0.945718,
Naive Bayes,0.948379,0.951795
Support vector machine,0.945718,
K-means,0.190142,
ECM,0.896861,
DeepMatcher,0.983851,0.990674


## Conclusion
Our analysis seems to indicate that machine learning approaches can be beneficial in the context of entity matching, but that the question whether deep learning really outperforms more traditional M models can be debated. Much will probably depend on the nature of the dataset, and the amount of time that one wishes to invest in labeling data and training the model. An interesting observation is that at least on our not too complicated dataset, the unsupervised ECM method also shows a very decent performance, which indicates that one can already gain something from machine learning without labeling the data.

Further research could be geared towards the effect of feature engineering on the methods (DeepMatcher used other features than PRLT) and on the question to what extent our simple heuristic was a good proxy to traditional programming methods implemented in existing entity matching systems.